In [1]:
# Librerias
import cv2
import os
import random

In [4]:
def create_directories(base_path, person_name):
    # Crear las carpetas de train y validation
    preprocessed_path = os.path.join(base_path, 'preprocessed')
    train_dir = os.path.join(preprocessed_path, 'train', person_name)
    test_dir = os.path.join(preprocessed_path, 'test', person_name)

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    print(f'Carpetas creadas: {train_dir} - {test_dir}')
    return train_dir, test_dir

def load_haar_cascade(cascade_path):
    face_classifier = cv2.CascadeClassifier(cascade_path)
    if face_classifier.empty():
        print("Error al cargar el clasificador Haar Cascade.")
        return None
    return face_classifier

def capture_faces(video_path, preprocessed_path, face_classifier, max_images=400):
    cap = cv2.VideoCapture(video_path)
    count = 0
    captured_faces = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Revisar los parametros de la detección porque puede variar segun la persona
        faces = face_classifier.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=4)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            face = frame[y:y + h, x:x + w]
            face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_CUBIC)
            captured_faces.append(face)
            count += 1
            
        if count == max_images:
            break
        
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return captured_faces

def split_dataset(captured_faces, train_dir, test_dir):
    total_images = len(captured_faces)
    
    if total_images == 0:
        print("No hay imágenes para dividir.")
        return

    # Mezclar aleatoriamente las imágenes
    random.shuffle(captured_faces)

    # Dividir el conjunto en 90% para entrenamiento y 10% para test
    split_index = int(total_images * 0.9)
    train_faces = captured_faces[:split_index]
    test_faces = captured_faces[split_index:]
    
    # Saber cuántas imágenes ya existen en las carpetas
    crops_train = len(os.listdir(train_dir))
    crops_test = len(os.listdir(test_dir))


    # Guardar las imágenes en las carpetas correspondientes SIN sobreescribir
    for i, face in enumerate(train_faces):
        filename = f'face_{crops_train + i}.jpg'
        cv2.imwrite(os.path.join(train_dir, filename), face)

    for i, face in enumerate(test_faces):
        filename = f'face_{crops_test + i}.jpg'
        cv2.imwrite(os.path.join(test_dir, filename), face)

    print('Distribución de imágenes completada.')

In [ ]:
def main():
    # Diccionario de personas y sus respectivos videos
    personas_videos = {
        'Abir Ahmed': ['Abir1.mp4', 'Abir2.mp4'],
        'Adriana Sanchez': ['AdriSa1.mp4', 'AdriSa2.mp4'],
        'Adriana Solanilla': ['AdriSo1.mp4', 'AdriSo2.mp4'],
        'Amy Olivares': ['Amy1.mp4', 'Amy2.mp4'],
        'Blas de Leon': ['Blas1.mp4', 'Blas2.mp4'],
        'Carlos Beitia': ['CarlosB1.mp4', 'CarlosB2.mp4'],
        'Carlos Hernandez': ['CarlosH1.mp4', 'CarlosH2.mp4'],
        'Cesar Rodriguez': ['Cesar1.mp4', 'Cesar2.mp4'],
        'David Rodriguez': ['David1.mp4', 'David2.mp4'],
        'Javier Bustamante': ['Javier1.mp4', 'Javier2.mp4'],
        'Jeremy Sanchez': ['Jeremy1.mp4', 'Jeremy2.mp4'],
        'Jonathan Peralta': ['Jonathan1.mp4', 'Jonathan2.mp4'],
        'Kevin Rodriguez': ['Kevin1.mp4', 'Kevin2.mp4'],
        'Mahir Arcia': ['Mahir1.mp4', 'Mahir2.mp4'],
        'Michael Jordan': ['Michael1.mp4', 'Michael2.mp4'],
        'Maria Donadio': ['Teresa1.mp4', 'Teresa2.mp4'],
        'Alejandro Tulipano': ['Tulipano1.mp4', 'Tulipano2.mp4'],
    }

    base_path = '../data'  # Ruta base
    haar_cascade_path = '../models/haarcascade_frontalface_alt.xml'

    # Cargar el clasificador una sola vez
    face_classifier = load_haar_cascade(haar_cascade_path)
    if face_classifier is None:
        print("No se pudo cargar el clasificador Haar Cascade.")
        return

    # Iterar por cada persona
    for person_name, videos in personas_videos.items():
        print(f"\nProcesando a {person_name}...")

        train_dir, test_dir = create_directories(base_path, person_name)

        for video_name in videos:
            video_path = os.path.join(base_path, 'crudo', video_name)

            if not os.path.exists(video_path):
                print(f"Video no encontrado !!: {video_path}")
                continue

            print(f"Procesando video: {video_name}")

            # Captura de rostros
            captured_faces = capture_faces(video_path, train_dir, face_classifier)

            if len(captured_faces) > 0:
                print(f'Se han capturado {len(captured_faces)} rostros del video {video_name}.')
                
                # Dividir en train/test
                split_dataset(captured_faces, train_dir, test_dir)
            else:
                print(f'No se capturaron rostros en {video_name}.')

if __name__ == "__main__":
    main()


In [ ]:
# Funcion Antigua (El procesamiento de videos era manual)

def main():
    # Nombre de la persona
    person_name = 'Nombre Apellido'  # Cambiar por el nombre de la persona

    # Rutas de carpetas
    base_path = '../data'  # Ajusta la ruta si es necesario
    video_path = os.path.join(base_path, 'crudo/Alumno 1.mp4')  # Ruta del video
    train_dir, test_dir = create_directories(base_path, person_name)  # Rutas para almacenar los rostros

    # Cargar el clasificador Haar Cascade
    haar_cascade_path = '../models/haarcascade_frontalface_alt.xml'
    face_classifier = load_haar_cascade(haar_cascade_path)

    if face_classifier is not None:
        # Captura de rostros
        captured_faces = capture_faces(video_path, train_dir, face_classifier)

        if len(captured_faces) > 0:
            print(f'Se han capturado {len(captured_faces)} rostros.')
            
            # Dividir el conjunto de datos en train y test
            split_dataset(captured_faces, train_dir, test_dir)
        else:
            print('No se han capturado rostros.')

if __name__ == "__main__":
    main()